### Edgar Moises Hernandez Gonzalez
#### 07/02/20-08/02/20
#### Clasificacion de imagenes motoras en EEG con CNN-1D
##### Caracteristicas = RAW
##### Optimizar hiperparametros con busqueda de cuadricula
##### Validacion cruzada de 5

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

In [ ]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A01T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A01E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA01T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA01E.csv", header=None)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)
(288, 1)


In [ ]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

print(x.shape)
print(y.shape)

(576, 22000)
(576, 1)


In [ ]:
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [ ]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [ ]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
x = StandardScaler().fit_transform(x)

In [ ]:
#codificar etiquetas con valores de 0 a nClases-1
y = LabelEncoder().fit_transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#convertir de 2D a 3D
x = x.reshape((n_samples, 22, 1000))

print(x.shape)

(576, 22, 1000)


In [ ]:
x = np.transpose(x, (0,2,1))

print(x.shape)

(576, 1000, 22)


In [ ]:
def CNN_1D(n_filtros_1, n_filtros_2):
  model = Sequential()
  model.add(Conv1D(n_filtros_1, 3, activation='relu', input_shape=(1000,22)))
  model.add(MaxPool1D(2))
  model.add(Conv1D(n_filtros_2, 11, activation='relu'))
  model.add(MaxPool1D(2))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(n_classes, activation='softmax'))

  optimizer = Adam(lr=1e-6)
  model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
clf = KerasClassifier(build_fn = CNN_1D, epochs=400, batch_size=18, verbose=0)

In [ ]:
n_filtros_1 = [8, 16, 32]
#tamano_filtros_1 = [3, 11]
n_filtros_2 = [8, 16, 32]
#tamano_filtros_2 = [3, 11]
hiperparametros = dict(n_filtros_1=n_filtros_1,
                       n_filtros_2=n_filtros_2)

In [ ]:
3 * 3

9

In [ ]:
model = GridSearchCV(clf, hiperparametros, n_jobs=-1, cv=5)

In [ ]:
model.fit(x, y)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


GridSearchCV(cv=5, error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f24584e1b00>,
             iid='deprecated', n_jobs=-1,
             param_grid={'n_filtros_1': [8, 16, 32],
                         'n_filtros_2': [8, 16, 32]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
print(model.best_params_)
print(model.best_score_)

{'n_filtros_1': 32, 'n_filtros_2': 32}
0.310704642534256


In [ ]:
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("%0.4f (+/-%0.04f) for %r"
        % (mean, std * 2, params))

0.2483 (+/-0.0498) for {'n_filtros_1': 8, 'n_filtros_2': 8}
0.2709 (+/-0.0463) for {'n_filtros_1': 8, 'n_filtros_2': 16}
0.2483 (+/-0.0307) for {'n_filtros_1': 8, 'n_filtros_2': 32}
0.3021 (+/-0.0468) for {'n_filtros_1': 16, 'n_filtros_2': 8}
0.2448 (+/-0.0558) for {'n_filtros_1': 16, 'n_filtros_2': 16}
0.2655 (+/-0.0718) for {'n_filtros_1': 16, 'n_filtros_2': 32}
0.2570 (+/-0.0509) for {'n_filtros_1': 32, 'n_filtros_2': 8}
0.2901 (+/-0.0789) for {'n_filtros_1': 32, 'n_filtros_2': 16}
0.3107 (+/-0.0504) for {'n_filtros_1': 32, 'n_filtros_2': 32}


In [ ]:
np.sort(means)

array([0.24481259, 0.24826087, 0.24827586, 0.2570015 , 0.26553224,
       0.27085457, 0.29005997, 0.30206896, 0.31070464])